In [21]:
#sample prediction
from fast_bert.prediction import BertClassificationPredictor

MODEL_PATH = '/home/sujitnashik/Downloads/model_out'
LABEL_PATH= '/home/sujitnashik/Downloads/nlp_data/'
predictor = BertClassificationPredictor(
				model_path=MODEL_PATH,
				label_path=LABEL_PATH,
				multi_label=False,
				model_type='bert',
				do_lower_case=False,
				device=None)




texts = [
	"this is the first text",
	"this is complex text"
	]

multiple_predictions = predictor.predict_batch(texts)
print(multiple_predictions)

[[('neutral', 0.8142256140708923), ('joy', 0.16540467739105225), ('surprise', 0.01722590997815132), ('anger', 0.002016712911427021), ('sadness', 0.000653256953228265), ('fear', 0.00027571909595280886), ('disgust', 0.0001981032983167097)], [('neutral', 0.9364937543869019), ('joy', 0.046546194702386856), ('surprise', 0.012686217203736305), ('anger', 0.0023554586805403233), ('sadness', 0.0007997110951691866), ('fear', 0.0007758369320072234), ('disgust', 0.0003428677446208894)]]


In [38]:
#prediction final
from collections import defaultdict
import pandas as pd
from fast_bert.prediction import BertClassificationPredictor

MODEL_PATH = '/home/sujitnashik/Downloads/model_out'
LABEL_PATH = '/home/sujitnashik/Downloads/nlp_data/'
labels_list = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']

predictor = BertClassificationPredictor(
    model_path=MODEL_PATH,
    label_path=LABEL_PATH,
    multi_label=False,
    model_type='bert',
    do_lower_case=False,
    device=None)

# Define a mapping of original labels to positive, negative, and neutral
emotion_mapping = {
    'anger': 'negative', 'disgust': 'negative', 'fear': 'negative',
    'joy': 'positive', 'sadness': 'negative', 'surprise': 'ambiguous',
    'neutral': 'neutral'
}# Define the data as a string
data = """
Original Dialogue:
Mike: Do you know where Tomas is from?
Jenny: Eastern Europe I believe
Mike: sure, but what country exactly
Mike: I heard him speaking English today with Kamil, so I think he's not Polish
Jack: Really? I was sure he was Polish
Kyle: He's from Slovenia
Mike: oh, how cute, how do you know?
Kyle: We talked many times about Slovenia and his home town
Mike: Which is?
Kyle: Bled I think, close to the Alps
Jack: and why do you find Slovenia cute? hahaha
Mike: I think he's the only Slovenian in the company now
Jack: true, quite exotic

Reference Summary:
Mike, Jenny and Jack wonder where Tomas is from. Kyle is sure Tomas is from Slovenia. Mike thinks Tomas is now the only Slovenian in the company.

Model-generated Summary:
[{'summary_text': "Tomas is from Eastern Europe, but he's not Polish. He's from Slovenia and is the only Slovenian in the company now."}]
"""

# Split the data into parts and then split the dialogue into lines
parts = data.split('\n\n')

original_dialogue_lines = parts[0].split('\n')[1:]  # Skip the title and split into lines

emotion_counts = defaultdict(lambda: {'positive': 0, 'negative': 0, 'neutral': 0, 'ambiguous'  : 0})

# Process each line of dialogue
for line in original_dialogue_lines:
    if ": " in line:
        speaker, text = line.split(": ", 1)
        predictions = predictor.predict(text)
        top_label = predictions[0][0] if predictions else 'neutral'
        mapped_label = emotion_mapping.get(top_label, 'neutral')
        print(f"{speaker}: Text: '{text}', Prediction: '{top_label}', Mapped Emotion: '{mapped_label}'")
        emotion_counts[speaker][mapped_label] += 1

# Process the summary
summary_text = parts[1].split('\n')[1]
summary_prediction = predictor.predict(summary_text)
summary_top_label = summary_prediction[0][0] if summary_prediction else 'neutral'
summary_mapped_label = emotion_mapping.get(summary_top_label, 'neutral')
print(f"\nSummary: '{summary_text}', Summary Emotion: {summary_mapped_label}")
emotion_df = pd.DataFrame(emotion_counts).fillna(0).astype(int).transpose()

model_generated_summary = eval(parts[2].split('\n')[1])[0]["summary_text"]
model_summary_prediction = predictor.predict(model_generated_summary)
model_summary_top_label = model_summary_prediction[0][0] if model_summary_prediction else 'neutral'
model_summary_mapped_label = emotion_mapping.get(model_summary_top_label, 'neutral')
print(f"\nModel-generated Summary: '{model_generated_summary}', Summary Emotion: {model_summary_mapped_label}")
print('\n\n')


emotion_df = pd.DataFrame(emotion_counts).fillna(0).astype(int).transpose()


def find_predominant_sentiment(row):
    max_value = row.max()
    predominant_sentiments = row[row == max_value].index.tolist()
    return ', '.join(predominant_sentiments) if len(predominant_sentiments) > 1 else predominant_sentiments[0]

# Apply this function to each row
emotion_df['Overall Sentiment'] = emotion_df.apply(find_predominant_sentiment, axis=1)

# Convert DataFrame to HTML table with borders
emotion_df_html = emotion_df.to_html(border=1)

# Display the HTML table in Jupyter Notebook
HTML(emotion_df_html)





Mike: Text: 'Do you know where Tomas is from?', Prediction: 'surprise', Mapped Emotion: 'ambiguous'
Jenny: Text: 'Eastern Europe I believe', Prediction: 'joy', Mapped Emotion: 'positive'
Mike: Text: 'sure, but what country exactly', Prediction: 'joy', Mapped Emotion: 'positive'
Mike: Text: 'I heard him speaking English today with Kamil, so I think he's not Polish', Prediction: 'neutral', Mapped Emotion: 'neutral'
Jack: Text: 'Really? I was sure he was Polish', Prediction: 'joy', Mapped Emotion: 'positive'
Kyle: Text: 'He's from Slovenia', Prediction: 'neutral', Mapped Emotion: 'neutral'
Mike: Text: 'oh, how cute, how do you know?', Prediction: 'joy', Mapped Emotion: 'positive'
Kyle: Text: 'We talked many times about Slovenia and his home town', Prediction: 'neutral', Mapped Emotion: 'neutral'
Mike: Text: 'Which is?', Prediction: 'surprise', Mapped Emotion: 'ambiguous'
Kyle: Text: 'Bled I think, close to the Alps', Prediction: 'neutral', Mapped Emotion: 'neutral'
Jack: Text: 'and why do

,positive,negative,neutral,ambiguous,Overall Sentiment
Mike,2,0,2,2,"positive, neutral, ambiguous"
Jenny,1,0,0,0,positive
Jack,3,0,0,0,positive
Kyle,0,0,3,0,neutral
